# Test speed_up_new_dff branch
- Matt has improved new_dff calculation codes by implementing pandas rolling
- Test the speed and compare the results with previous codes (_SLOW)

In [3]:
from brain_observatory_qc.pipeline_dev import calculate_new_dff as cnd
from brain_observatory_qc.data_access import from_lims

## Testing with new calculations

## Testing with previous results

In [10]:
oeid = 1167237079
np_corrected_df = cnd.get_np_corrected_df(oeid)
np_corrected_df.head()

,cell_specimen_id,cell_roi_id,np_corrected,np_not_corrected,r,r_out_of_range
0,1263873779,1262574825,"[322.33874682846107, 317.5853936385364, 307.04...","[453.5748299319728, 444.7244897959184, 439.869...",0.376,0
1,1263873773,1262574822,"[467.1923504286358, 313.32750179403973, 368.01...","[503.3114754098361, 472.8274785323966, 485.845...",0.309,0
2,1263873744,1262574765,"[673.7378439892402, 714.0467092902954, 761.412...","[486.80167597765364, 455.0817039106145, 452.64...",0.208,0
3,1263873738,1262574748,"[661.9483343439027, 682.7125206691157, 635.464...","[473.20415879017014, 470.58349086326405, 465.5...",0.288,0
4,1263873735,1262574738,"[476.038245139476, 364.8985731191886, 472.6741...","[518.200338123415, 494.3474218089603, 485.4615...",0.276,0


In [11]:
inactive_kernel_size = 30
inactive_percentile = 10
frame_rate, timestamps = cnd.get_correct_frame_rate(oeid)
long_filter_length = int(round(frame_rate * 60 * inactive_kernel_size))
short_filter_length = int(round(frame_rate * 60 * 10))  # 10 min


c:\users\jinho.kim\github\allensdk\allensdk\brain_observatory\behavior\data_objects\stimuli\presentations.py:48: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  presentations.index = pd.Int64Index(
c:\users\jinho.kim\github\allensdk\allensdk\brain_observatory\behavior\data_objects\eye_tracking\rig_geometry.py:233: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rig_geometry = pd.read_sql(query, lims_db.get_connection())
c:\users\jinho.kim\github\allensdk\allensdk\brain_observatory\behavior\data_objects\cell_specimens\cell_specimens.py:260: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQL

In [29]:
ci = 1
crid = np_corrected_df.iloc[ci]['cell_roi_id']
corrected_trace = np_corrected_df.iloc[ci]['np_corrected']

In [30]:
import time
import numpy as np

start = time.time()
new_dff_faster, _ = cnd.new_dff_each_cell(corrected_trace, crid, long_filter_length,
                                    inactive_percentile, short_filter_length, frame_rate)
end = time.time()
print(end - start)

start = time.time()
new_dff_slower, _ = cnd.new_dff_each_cell_SLOW(corrected_trace, crid, long_filter_length,
                                    inactive_percentile, short_filter_length, frame_rate)
end = time.time()
print(end - start)

diff = np.abs(new_dff_slower - new_dff_faster)
print(np.mean(diff))
print(np.mean(diff[diff>0]))
print(np.max(diff)/max(np.max(new_dff_slower), np.max(new_dff_faster)))
print(len(diff[diff>0])/len(diff))

0.12221813201904297
17.829222917556763
8.98686000714426e-06
2.7261048306521535e-05
0.00036012770175264115
0.3296593698854337
